In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [5]:
baseline = pd.read_excel(r"C:\Users\mrmik\OneDrive\Desktop\NSS\Python\Projects\Cycling_Capstone\notebooks\baseline_data.xlsx")

In [7]:
baseline

,Unnamed: 0,PCS race entry,Race Name,Year,Winner,GC speciality points,time_trial speciality points,sprint speciality points,climber speciality points,hills speciality points,...,Won How,date,profile_icon,profile_score,departure,arrival,average_temperature,start_time,race_datetime,race_coords
0,0,2017amstelgoldrace,Amstel Gold Race,2017,GILBERT Philippe,4215.0,1426.0,1094.0,3215.0,5140.0,...,Sprint à deux,2017-04-16,"Hills, flat finish",92.0,Maastricht,Berg en Terblijt,NaN,1900-01-01 11:26:00,2017-04-16 11:26:00,"[(50.87808, 5.71738), (50.87819, 5.71747), (50..."
1,1,2018amstelgoldrace,Amstel Gold Race,2018,VALGREN Michael,1188.0,453.0,84.0,955.0,1858.0,...,Sprint à deux,2018-04-15,"Hills, flat finish",103.0,Maastricht,Berg en Terblijt,NaN,1900-01-01 10:35:00,2018-04-15 10:35:00,"[(50.87808, 5.71738), (50.87819, 5.71747), (50..."
2,2,2019amstelgoldrace,Amstel Gold Race,2019,VAN DER POEL Mathieu,1119.0,968.0,1286.0,1042.0,3090.0,...,Sprint of small group,2019-04-21,"Hills, flat finish",108.0,Maastricht,Berg en Terblijt,NaN,1900-01-01 12:35:00,2019-04-21 12:35:00,"[(50.86287, 5.70027), (50.86307, 5.70027), (50..."
3,3,2021amstelgoldrace,Amstel Gold Race,2021,VAN AERT Wout,1171.0,2944.0,1886.0,2044.0,4221.0,...,Sprint of small group,2021-04-18,"Hills, flat finish",81.0,Valkenburg,Berg en Terblijt,NaN,1900-01-01 12:11:00,2021-04-18 12:11:00,"[(50.86699, 5.78322), (50.86697, 5.78322), (50..."
4,4,2022amstelgoldrace,Amstel Gold Race,2022,KWIATKOWSKI Michał,3670.0,3561.0,755.0,3304.0,3579.0,...,Sprint à deux,2022-04-10,"Hills, flat finish",112.0,Maastricht,Valkenburg,10.0,1900-01-01 10:26:00,2022-04-10 10:26:00,"[(50.86307, 5.70026), (50.86312, 5.70026), (50..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3,2019kuurne-brussel-kuurne,Kuurne-Brussel-Kuurne,2019,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
96,3,2020kuurne-brussel-kuurne,Kuurne-Brussel-Kuurne,2020,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
97,3,2021kuurne-brussel-kuurne,Kuurne-Brussel-Kuurne,2021,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
98,3,2022kuurne-brussel-kuurne,Kuurne-Brussel-Kuurne,2022,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN


In [21]:
race_names_add = {
    "2017 Il Lombardia": "race/il-lombardia/2017/result",
    "2018 Il Lombardia": "race/il-lombardia/2018/result",
    "2019 Il Lombardia": "race/il-lombardia/2019/result",
    "2020 Il Lombardia": "race/il-lombardia/2020/result",
    "2021 Il Lombardia": "race/il-lombardia/2021/result",
    "2022 Il Lombardia": "race/il-lombardia/2022/result",
    "2023 Il Lombardia": "race/il-lombardia/2023/result",
    "2017 Kuurne-Brussel-Kuurne": "race/kuurne-brussel-kuurne/2017/result",
    "2018 Kuurne-Brussel-Kuurne": "race/kuurne-brussel-kuurne/2018/result",
    "2019 Kuurne-Brussel-Kuurne": "race/kuurne-brussel-kuurne/2019/result",
    "2020 Kuurne-Brussel-Kuurne": "race/kuurne-brussel-kuurne/2020/result",
    "2021 Kuurne-Brussel-Kuurne": "race/kuurne-brussel-kuurne/2021/result",
    "2022 Kuurne-Brussel-Kuurne": "race/kuurne-brussel-kuurne/2022/result",
    "2023 Kuurne-Brussel-Kuurne": "race/kuurne-brussel-kuurne/2023/result",
}

In [17]:
from procyclingstats import Stage
race_names_add 
data = []

for race_name, race_url in race_names_add.items():
    try:
        stage = Stage(race_url)
        won_how = stage.won_how() 
        data.append({"Race Name": race_name, "Won How": won_how})
    except Exception as e:
        print(f"Error fetching {race_name}: {e}")
        data.append({"Race Name": race_name, "Won How": "Error fetching data"})

df = pd.DataFrame(data)
df.to_excel("wonhow_2.xlsx", index=False)

In [23]:
from procyclingstats import Stage
race_names_add
data = []

for race_name, race_url in race_names_add.items():
    try:
        stage = Stage(race_url)
        avg_speed = stage.avg_speed_winner()
        results = stage.results("rank", "rider_name", "team_name", "time")

        while len(results) < 5:
            results.append({"rider_name": "N/A", "team_name": "N/A", "time": "N/A"})

        top_5 = results[:5]  
        winner_time = top_5[0]["time"]
        time_gaps = [f"{rider['rider_name']} - {rider['time']}" for rider in top_5[1:]]

        data.append({
            "Race": race_name,
            "Winner": top_5[0]["rider_name"],
            "Team": top_5[0]["team_name"],
            "Avg Speed (km/h)": avg_speed if avg_speed else "N/A",
            "Won How": won_how,
            "2nd Place": top_5[1]["rider_name"],
            "3rd Place": top_5[2]["rider_name"],
            "4th Place": top_5[3]["rider_name"],
            "5th Place": top_5[4]["rider_name"],
            "Time Gaps": ", ".join(time_gaps),
        })
        
    except Exception as e:
        print(f"Error fetching {race_name}: {e}")
        data.append({
            "Race": race_name,
            "Winner": "Error fetching data",
            "Team": "N/A",
            "Avg Speed (km/h)": "N/A",
            "Won How": "N/A",
            "2nd Place": "N/A",
            "3rd Place": "N/A",
            "4th Place": "N/A",
            "5th Place": "N/A",
            "Time Gaps": "N/A",
        })

df = pd.DataFrame(data)
df.to_excel("top5_finishers2.xlsx", index=False)

In [27]:
for race_name, race_url in race_names_add.items():
    try:
        stage = Stage(race_url)

        profile_icon = stage.profile_icon()  
        profile_score = stage.profile_score() 

        data.append({
            "Race": race_name,
            "Profile Icon": profile_icon if profile_icon else "N/A",
            "Profile Score": profile_score if profile_score else "N/A"
        })

        print(f"{race_name} - Profile: {profile_icon}, Score: {profile_score}")

    except Exception as e:
        print(f"Error fetching {race_name}: {e}")
        data.append({
            "Race": race_name,
            "Profile Icon": "Error fetching",
            "Profile Score": "Error fetching"
        })

df = pd.DataFrame(data)
df.to_excel("pcs_race_profiles2.xlsx", index=False)

2017 Il Lombardia - Profile: p2, Score: 279
2018 Il Lombardia - Profile: p2, Score: 257
2019 Il Lombardia - Profile: p4, Score: 266
2020 Il Lombardia - Profile: p2, Score: 260
2021 Il Lombardia - Profile: p2, Score: 210
2022 Il Lombardia - Profile: p2, Score: 278
2023 Il Lombardia - Profile: p2, Score: 206
2017 Kuurne-Brussel-Kuurne - Profile: p1, Score: 22
2018 Kuurne-Brussel-Kuurne - Profile: p2, Score: 22
2019 Kuurne-Brussel-Kuurne - Profile: p2, Score: 40
2020 Kuurne-Brussel-Kuurne - Profile: p2, Score: 38
2021 Kuurne-Brussel-Kuurne - Profile: p2, Score: 38
2022 Kuurne-Brussel-Kuurne - Profile: p2, Score: 45
2023 Kuurne-Brussel-Kuurne - Profile: p2, Score: 43


In [35]:
from procyclingstats import Stage
race_names_add
all_race_data = []

for race_name, race_url in race_names_add.items():
    try:
        stage = Stage(race_url)

        race_data = {
            "PCS race entry": race_name.lower(),
            "date": stage.date(),
            "profile_icon": stage.profile_icon(),
            "profile_score": stage.profile_score(),
            "departure": stage.departure(),
            "arrival": stage.arrival(),
            "average_temperature": stage.avg_temperature(),
            "start_time": stage.start_time(),
        }

        all_race_data.append(race_data)

        print(f"Successfully retrieved {race_name}")

    except Exception as e:
        print(f"Error fetching {race_name}: {e}")

df = pd.DataFrame(all_race_data)
df.to_excel("baseline_adds.xlsx")

Successfully retrieved 2017 Il Lombardia
Successfully retrieved 2018 Il Lombardia
Successfully retrieved 2019 Il Lombardia
Successfully retrieved 2020 Il Lombardia
Successfully retrieved 2021 Il Lombardia
Successfully retrieved 2022 Il Lombardia
Successfully retrieved 2023 Il Lombardia
Successfully retrieved 2017 Kuurne-Brussel-Kuurne
Successfully retrieved 2018 Kuurne-Brussel-Kuurne
Successfully retrieved 2019 Kuurne-Brussel-Kuurne
Successfully retrieved 2020 Kuurne-Brussel-Kuurne
Successfully retrieved 2021 Kuurne-Brussel-Kuurne
Successfully retrieved 2022 Kuurne-Brussel-Kuurne
Successfully retrieved 2023 Kuurne-Brussel-Kuurne


In [87]:
from procyclingstats import Rider

winners_add = {
    "NIBALI Vincenzo": "rider/vincenzo-nibali",
    "PINOT Thibaut": "rider/thibaut-pinot",
    "MOLLEMA Bauke": "rider/bauke-mollema",
    "FUGLSANG Jakob": "rider/jakob-fuglsang",
    "POGAČAR Tadej": "rider/tadej-pogačar",
    "SAGAN Peter": "rider/peter-sagan",
    "GROENEWEGEN Dylan": "rider/dylan-groenewegen",
    "JUNGELS Bob": "rider/bob-jungels",
    "ASGREEN Kasper": "rider/kasper-asgreen",
    "PEDERSEN Mads": "rider/mads-pedersen",
    "JAKOBSEN Fabio": "rider/fabio-jakobsen",
    "BENOOT Tiesj": "rider/tiesj-benoot"
}

data = []

for rider_name, rider_url in winners_add.items():
    try:
        rider_obj = Rider(rider_url)
        points = rider_obj.points_per_speciality()
        data.append({'Rider': rider_name, **points})
    except Exception as e:
        print(f"Error fetching {rider_name}: {e}")
        
df_api = pd.DataFrame(data)
df_api.to_excel("rider_points.xlsx", index=False)

In [107]:
oneday_pps = {
    "GILBERT Philippe": "rider/philippe-gilbert",
    "VALGREN Michael": "rider/michael-valgren-andersen",
    "VAN DER POEL Mathieu": "rider/mathieu-van-der-poel",
    "VAN AERT Wout": "rider/wout-van-aert",
    "KWIATKOWSKI Michał": "rider/michal-kwiatkowski",
    "POGAČAR Tadej": "rider/tadej-pogacar",
    "COLBRELLI Sonny": "rider/sonny-colbrelli",
    "WELLENS Tim": "rider/tim-wellens",
    "ALAPHILIPPE Julian": "rider/julian-alaphilippe",
    "PIDCOCK Thomas": "rider/thomas-pidcock",
    "SHEFFIELD Magnus": "rider/magnus-sheffield",
    "GODON Dorian": "rider/dorian-godon",
    "LAMPAERT Yves": "rider/yves-lampaert",
    "VAN BAARLE Dylan": "rider/dylan-van-baarle",
    "LAPORTE Christophe": "rider/christophe-laporte",
    "ŠTYBAR Zdeněk": "rider/zdenek-stybar",
    "ASGREEN Kasper": "rider/kasper-asgreen",
    "VAN AVERMAET Greg": "rider/greg-van-avermaet",
    "SAGAN Peter": "rider/peter-sagan",
    "KRISTOFF Alexander": "rider/alexander-kristoff",
    "PEDERSEN Mads": "rider/mads-pedersen",
    "GIRMAY Biniam": "rider/biniam-girmay",
    "VALVERDE Alejandro": "rider/alejandro-valverde",
    "HIRSCHI Marc": "rider/marc-hirschi",
    "TEUNS Dylan": "rider/dylan-teuns",
    "JUNGELS Bob": "rider/bob-jungels",
    "FUGLSANG Jakob": "rider/jakob-fuglsang",
    "EVENEPOEL Remco": "rider/remco-evenepoel",
    "NIBALI Vincenzo": "rider/vincenzo-nibali",
    "STUYVEN Jasper": "rider/jasper-stuyven",
    "MOHORIČ Matej": "rider/matej-mohoric",
    "BALLERINI Davide": "rider/davide-ballerini",
    "TERPSTRA Niki": "rider/niki-terpstra",
    "BETTIOL Alberto": "rider/alberto-bettiol",
    "KITTEL Marcel": "rider/marcel-kittel",
    "JAKOBSEN Fabio": "rider/fabio-jakobsen",
    "EWAN Caleb": "rider/caleb-ewan",
    "PHILIPSEN Jasper": "rider/jasper-philipsen",
    "BENOOT Tiesj": "rider/tiesj-benoot",
    "PINOT Thibaut": "rider/thibaut-pinot",
    "MOLLEMA Bauke": "rider/bauke-mollema",
    "GROENEWEGEN Dylan": "rider/dylan-groenewegen"
}

In [109]:
from procyclingstats import Rider

oneday_pps
data = []

for rider_name, rider_url in oneday_pps_winners.items():
    try:
        rider_obj = Rider(rider_url)
        points = rider_obj.points_per_speciality()
        data.append({'Rider': rider_name, **points})
    except Exception as e:
        print(f"Error fetching {rider_name}: {e}")
        
df_api = pd.DataFrame(data)
df_api.to_excel("oneday_pps.xlsx", index=False)

Error fetching VALGREN Michael: HTML from given URL is invalid: 'https://www.procyclingstats.com/rider/michael-valgren'
